## Import the Library

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras 
import imutils
from imutils.video import VideoStream

## Load the model

In [2]:
# model that we made for face mask detection
model = load_model(r'F:\face_detection_model (1).h5')

# Load the model for face detection
prototxtPath = r'F:\deploy.prototxt'
weightsPath = r'F:\res10_300x300_ssd_iter_140000.caffemodel'

In [3]:
net = cv2.dnn.readNet(prototxtPath, weightsPath)

## function for detecting mask

In [4]:
def detect_and_predict_mask(frame, faceModel, maskModel):
    # height and width of the frame
    (h, w) = frame.shape[:2]
    # pass the frame to the blobfomImage for image processing like resizing and mean subtraction
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),(104.0, 177.0, 123.0))
    # pass the blob to the face detection model and predict the faces
    faceModel.setInput(blob)
    detections = faceModel.forward()
    # array defining no of faces in the frames, their corresponding location of faces and predictions
    faces = []
    locs = []
    preds = []
    # loop over the detection
    for i in range(0, detections.shape[2]):
        # store the probability associated with detection
        confidence = detections[0, 0, i, 2]
        # take only that part whose probability of detecting face is greater that 50%
        if confidence > 0.5 :
            # x,y coordinates for bounding box for the face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            # ensure bounding box fall within dimension of frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            # extract the face from image
            face = frame[startY:endY, startX:endX]
            # convert image from BGR to RGB
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            # Resize the face to (224,224)
            face = cv2.resize(face, (224, 224))
            # convert the face into array of pixels
            face = img_to_array(face)
            # preprocess the face
            face = preprocess_input(face)
            # add the faces and bounding boxes to the defined list
            faces.append(face)
            locs.append((startX, startY, endX, endY))
        # predict when atleast one face is detected
        if len(faces) > 0:\
            # convert the faces into the array and pass it to the model for mask detection
            faces = np.array(faces, dtype="float32")
            preds = maskModel.predict(faces, batch_size=32)
    # return the face loaction and corresponding prediction
    return (locs, preds)

## Detecting Face mask from video stream

In [7]:
vs= VideoStream(src=0).start()

while True:
    # take the frames from the video and resize it to the width of 400
    frame=vs.read()
    frame=imutils.resize(frame,width=400)
    # pass the frames to the function define for face and mask detection and predict the outcome for mask or no mask 
    (locs,preds)=detect_and_predict_mask(frame,net,model)
    #loop over the detected face locations and their corrosponding loactions    
    for (box,pred) in zip(locs,preds):
        (startX,startY,endX,endY)=box
        (mask,withoutMask)=pred        
        #determine the class label and color we will use to draw the bounding box and text
        if( mask > withoutMask):
            label = 'Mask'
            color = (0,255,0)
        else:
            label = 'No Mask'
            color = (0,0,255)        
        #display the label and bounding boxes
        cv2.putText(frame,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)       
        cv2.rectangle(frame,(startX,startY),(endX,endY),color,2)
    #show the output frame
    cv2.imshow("Frame",frame)
    key=cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break
        
cv2.destroyAllWindows()
vs.stream.release()
vs.stop()